# <span style="color:lightcoral; font-family:freestyle script; font-size:4em">SLOSH till relationsdatabas</span> <br><span style="color:black; font-family:freestyle script; font-size:3em">Förbereda färdig metadata för SQLite</span>
***

# Innehållsförteckning  
1. [Introduktion](#1)  
2. [Kolumner](#2)  
    2.1 [Enkät](#2.1)  
    2.2 [Itemnr](#2.2)  
    2.3 [Frekvens](#2.3)
3. [Tabeller](#3)  
    3.1 [Variabler](#3.1)  
    3.2 [Frekvenstabell](#3.2)  
    3.3 [Enkättexter](#3.3)  
    3.4 [Kodlistor](#3.4)  
    3.5 [Svarstexter](#3.5)  
    3.6 [Kodlistor_stor](#3.6)

***

# <a id = "1">1. Introduktion</a>  
I vad som följer kommer kursiv stil endast användas för att benämna *tabeller*. Fetstil kommer endast användas för att benämna **kolumner**, och värden i en kolumn kommer att vara i stil med (t.ex.) `SLOSHW08`, `Missing`, `142`. Även datatyper (t.ex. `str` eller `float64`) och pythonobjekt (t.ex. listor) kommer att ha denna formatering. Den metadata som redan är dokumenterad enligt Vetenskapsrådets riktlinjer ligger i excelfilen *metadata*. Några av tabellerna i relationsdatabsen innehåller kolumner som inte finns i denna fil. Dessa kolumner skapas i avsnittet [Kolumner](#2). Därefter kommer följande tabeller att skapas:  

|Variabler|Frekvenstabell|Enkättexter|Kodlistor|Svarstexter|Kodlistor_stor|  
|:---:|:---:|---|---|---|---|
|Variabel|V_id|Enkättext|Kodlista|Svarstext|K_id|  
|Enkät|Variabel|Källa|From|From|Kodlista|  
|Beskrivning|Enkät|-|Tom|Tom|Kod|  
|Itemnr|Kod|-|-|-|Svarstext|  
|Enkättext|Frekvens|-|-|-|-|  
|Kodlista|-|-|-|-|-|  
  
  
Relationsdatabasen kommmer att innehålla ytterligare fyra tabeller. Dessa är *Begrepp*, *Begrepp_stor*, *Tidsserie* samt *Tidsserie_stor*. För tillfället finns ingen data till dessa tabeller i *metadata* och därför kommer de inte skapas här. 

In [13]:
import numpy as np
import pandas as pd

In [14]:
df = pd.read_excel('Metadata.xlsx')

För att ta fram *Kodlista_stor* används `missing` i **text**. I *metadata* stavas `Missing` ibland med "M" och ibland med "m"(`missing`). Python läser detta som olika `str`, därför ersätts liten bokstav med stor.

In [3]:
df.replace('missing', 'Missing', inplace = True)

In [4]:
df.columns

Index(['Namn', 'Tidsserie', 'Beskrivning', 'Enkät', 'Dubbelkodning',
       'Enkättext', 'From', 'Tom', 'Kodlista', 'From.1', 'Tom.1', 'Kod',
       'Text', 'From.2', 'Tom.2', 'Koncept 1', 'Koncept 2', 'Instrument 1',
       'Instrument 2', 'Källa', 'SLOSHW06', 'frek', 'SLOSHW08', 'frek.1',
       'SLOSHW10', 'frek.2', 'SLOSHW12', 'frek.3', 'SLOSHW14', 'frek.4',
       'SLOSHW16', 'frek.5', 'SLOSHW18', 'frek.6', 'SLOSHNW06', 'frek.7',
       'SLOSHNW08', 'frek.8', 'SLOSHNW10', 'frek.9', 'SLOSHNW12', 'frek.10',
       'SLOSHNW14', 'frek.11', 'SLOSHNW16', 'frek.12', 'SLOSHNW18', 'frek.13'],
      dtype='object')

*** 
# <a id = "2">2. Kolumner</a>  
De kolumner som måste skapas från *metadata* är **Enkät**, **Itemnr** och **Frekvens**.  

***  
## <a id = "2.1">2.1 Enkät</a>  
Den här kolumnen ska säga vilket år (eller vilken våg) variabeln förekommer i. Samma kolumn ska säga huruvida variabeln finns för den arbetande- eller den icke-arbetande delen av SLOSH. Ett exempel på ett värde i kolumnen är `SLOSHW08`. Här står "W" för "working" ("non-working" förkortas "NW") och "08" för den andra SLOSH-vågen (som ägde rum 2008). Värdena i den här kolumnen är namn på (redan existerande) kolumner i *metadata*. Börja med att samla namnen på working-kolumner respektive nonworking-kolumner (från *metadata*) i två listor. Listorna kallas `WORKING` respektive `NONWORKING`.

In [5]:
WORKING = ['SLOSHW06', 'SLOSHW08', 'SLOSHW10', 'SLOSHW12', 'SLOSHW14', 'SLOSHW16', 'SLOSHW18']
NONWORKING = ['SLOSHNW06', 'SLOSHNW08', 'SLOSHNW10', 'SLOSHNW12', 'SLOSHNW14', 'SLOSHNW16', 'SLOSHNW18']

Kolumnerna som står för vågor i SLOSH (alltså t.ex. SLOSHW06) innehåller `NaN` ifall en given variabel inte finns med i vågen. Om variabeln istället *finns* kommer vågkolumnen innehålla en `str`. Varje variabel kan förekomma i högst två kolumner (en SLOSHW och en SLOSHNW). Ifall en given variabel förekommer i *endast en* kolumn, t.ex. **SLOSHW06**, ska variabelns värde i **Enkät** vara `SLOSHW06`.

In [6]:
for w,nw in zip(WORKING, NONWORKING):
    df.loc[(df[w].isna() == False) & (df[nw].isna() == True), ['Enkät']] = w
    df.loc[(df[nw].isna() == False) & (df[w].isna() == True), ['Enkät']] = nw

In [7]:
df['Enkät'].value_counts()

SLOSHW06       1210
SLOSHW08        889
w+n             748
SLOSHW10        669
SLOSHW12        579
SLOSHW14        567
SLOSHW16        517
SLOSHW18        426
SLOSHNW08       307
working         181
SLOSHNW12       160
SLOSHNW14       130
SLOSHNW16       106
SLOSHNW18        75
SLOSHNW10        41
non-working      34
SLOSHNW06        24
Name: Enkät, dtype: int64

Frekvenstabellen visar att det finns tre värden i **Enkät** för vilka tillhörande variabler förekommer i två vågkolumner. Detta är oväntat. Alla variabler som förekommer i två vågor ska egentligen ha **Enkät**-värdet `w+n`. Alltså har vi (någonstans) dokumneterat fel. Tillsvidare får dessa variabler, som borde vara `w+n`, **Enkät**-värdet `oklar`.

In [8]:
df.loc[(df['Enkät'] == 'working') | (df['Enkät'] == 'non-working'), ['Enkät']] = 'oklar'

Samla namn på alla variabler där **Enkät** $ = $ `oklar` så att de enkelt går att leta upp i excelfilen och rätta till.

In [55]:
a = [df.loc[i, 'Namn'] for i in df[df['Enkät'] == 'oklar'].index]

In [9]:
oklar = set([df.loc[i, 'Namn'] for i in df[df['Enkät'] == 'oklar'].index])

In [10]:
len(oklar)

43

In [48]:
oklar

{'capacphy_1',
 'capacpsy_1',
 'decbased_7',
 'decchang_7',
 'decconse_7',
 'decfolup_7',
 'decinfor_7',
 'decparts_7',
 'decright_7',
 'empltxt_2',
 'infoexer_3',
 'infoexer_4',
 'infoexer_5',
 'infoexer_6',
 'infoexer_7',
 'infofood_3',
 'infofood_4',
 'infofood_5',
 'infofood_6',
 'infofood_7',
 'interrup_1',
 'manalist_7',
 'pleasant_7',
 'sick1v_3',
 'sick1v_4',
 'sick1v_5',
 'sick1v_6',
 'sick1v_7',
 'sickchil_3',
 'sickchil_4',
 'sickchil_5',
 'sickchil_6',
 'sickchil_7',
 'sickmore_3',
 'sickmore_4',
 'sickmore_5',
 'sickmore_6',
 'sickmore_7',
 'sickwork_3',
 'stranswe_7',
 'strcompu_7',
 'strinterr_7',
 'strphone_7'}

  ***  
## <a id = "2.2">2.2 Itemnr</a>  
Nu passar **Enkät** in i relationsdatabasen (med undantag för de variabler som är `oklara`). **Itemnr** ska innehålla `str` som för närvarande ligger utspridda i vågkolumnerna i *metadata*. Ett värde i kolumnen kan t.ex. vara `A26_b`. Här är ett förenklat schema som visar relationen mellan **Itemnr** och vågkolumnerna i *metadata*:

In [11]:
pd.DataFrame([['A25', 'NaN', 'Nan', 'A25'], ['NaN', 'B23', 'Nan', 'B23'], ['NaN', 'NaN', 'C13', 'C13']], 
             columns = ['SLOSHW06', 'SLOSHNW10', 'SLOSHW16', 'Itemnr'])

,SLOSHW06,SLOSHNW10,SLOSHW16,Itemnr
0,A25,NaN,Nan,A25
1,NaN,B23,Nan,B23
2,NaN,NaN,C13,C13


Först skapas en ny kolumn som får namnet "Itemnr", kolumnen fylls med `NaN`.

In [12]:
df['Itemnr'] = np.nan

Proceduren för att välja ut rätt itemnummer illustreras i exemplet nedan.  
***  
  <span style="color:red">**Exempel**

In [13]:
d = pd.DataFrame([[1,2,3], [4,5,6], [1,8,9]], columns = ['A', 'B', 'C'])
d['test'] = 0

In [14]:
d

,A,B,C,test
0,1,2,3,0
1,4,5,6,0
2,1,8,9,0


In [15]:
d.loc[d['A'] == 1, ['test']] = d['B']
d

,A,B,C,test
0,1,2,3,2
1,4,5,6,0
2,1,8,9,8


<span style="color:red">**Exempel slut**  
***

Loopen nedan går igenom vågkolumnerna i *metadata*. Om en variabel har fått värdet `SLOSHW06` i **Enkät** kommer värdet för samma variabel i vågkolumnen **SLOSHW06** att bli variabelns värde i den nya kolumnen **Itemnr**.

In [16]:
for w, nw in zip(WORKING, NONWORKING):
    df.loc[df['Enkät'] == w, ['Itemnr']] = df[w]
    df.loc[df['Enkät'] == nw, ['Itemnr']] = df[nw]

Varabler med **Enkät** `oklar` ges **Itemr** `oklar`. 

In [17]:
df.loc[df['Enkät'] == 'oklar', ['Itemnr']] = 'oklar'

Gör en `value_counts` för att se att någon frekvenstabell i *metadata* hamnat på fel ställe. De sista sju värdena är definitivt inte itemnummer. Förmodligen har den här frekvenstabellen förskjutits någon kolumn...

In [47]:
df[df['Itemnr'] == 686]

,Namn,Tidsserie,Beskrivning,Enkät,Dubbelkodning,Enkättext,From,Tom,Kodlista,From.1,...,SLOSHNW12,frek.10,SLOSHNW14,frek.11,SLOSHNW16,frek.12,SLOSHNW18,frek.13,Itemnr,Frekvens
3678,keyboard_1,NaN,NaN,SLOSHW08,NaN,Fysisk arbetsmiljö @ Hur stor del av arbetstid...,2006,NaN,helatiden1,2006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,686,471.0


In [18]:
df['Itemnr'].value_counts()

oklar      215
27          47
29          41
7_b         37
4           33
7_a         32
7_c         30
11_b        29
2           28
31_d        28
11_a        28
31_c        27
51_c        26
51_b        26
19_a        26
47_a        25
28          25
51_a        25
41          24
20          24
7_e         23
25_b        23
7_f         23
7_d         23
7_g         23
25_a        23
46_a        22
28_b        22
28_a        22
7_i         22
          ... 
A44_a        3
A1_h         3
A1_g         3
A34          3
A44_b        3
A52_a        3
A45          2
A61_txt      2
3_txt        2
1_txt        2
A64          2
A54_txt      2
A50_txt      2
A20_txt      2
A63          2
A46          2
A47          2
49           2
A52_b        2
6_a_txt      2
58_a         2
61_a         2
A56_txt      2
686          1
638          1
915          1
1205         1
599          1
1009         1
89           1
Name: Itemnr, Length: 680, dtype: int64

***  
## <a id = "2.3">2.3 Frekvens</a>  
Kolumnen med frekvenser görs på ungefär samma sätt som **Itemnr**. I *metadata* ligger en frekvenskolumn intill varje vågkolumn. Exempelvis åtföljs **SLOSHW06** av en kolumn som rymmer frekvenser för de variabler som förekommer i **SLOSHW06**. De variabler som *inte* förekommer i **SLOSHW06** har värdet `NaN`i samma frekvenskolumn.  
  
Skapa först en ny kolumn med namnet **Frekvens** och fyll den med `NaN`.


In [19]:
df['Frekvens'] = np.nan

Skapa två listor, en för arbetande och en för icke-arbetande, och fyll dessa med namnen på frekvenskolumnerna från *metadata*.

In [20]:
WORKFREK = ['frek', 'frek.1', 'frek.2', 'frek.3', 'frek.4', 'frek.5', 'frek.6']
NONWORKFREK = ['frek.7', 'frek.8', 'frek.9', 'frek.10', 'frek.11', 'frek.12', 'frek.13']

Loopen nedan går igenom frekvenskolumnerna i *metadata*. De variabler som har frekvenser för högst en våg får detta värde i den nya frekvenskolumnen. De variabler som har frekvenser i två vågor, alltså de som i *metadata* är `w+n` samt ej är dubbelkodade, får tillsvidare `NaN` i den nya frekvenskolumnen. Anledningen är att det i avsnittet [Enkät](#2.2) antyds att vissa variabler är feldokumenterade.

In [21]:
for w, nw in zip(WORKFREK, NONWORKFREK):
    df.loc[(df[w].isna() == False) & (df[nw].isna() == True), ['Frekvens']] = df[w]
    df.loc[(df[nw].isna() == False) & (df[w].isna() == True), ['Frekvens']] = df[nw]

***  
# <a id = "3">3. Tabeller</a>  
I det här avnittet kommer tabellerna (förutom de som rör tidsserier och begrepp, se [introduktionen](#1)) till relationsdatabasen att skapas. Först tabellen *Variabler* (som är störst). Därefter *Frekvenstabeller*, *Enkättexter*, *Kodlistor*, *Svarstexter* och till sist *Kodlistor_stor*. 

***  
## <a id = "3.1">3.1 Variabler</a>  
Den här tabellen ska innehålla kolumnerna **Variabel**, **Enkät**, **Beskrivning**, **Itemnr**, **Enkättext** och **Kodlista**. Samtliga kolumner finns redan i *metadata* (numera `df`), dock kallas **Variabel** där för **Namn** och måste därför döpas om. 

In [22]:
Variabler = df[['Namn', 'Enkät', 'Beskrivning', 'Itemnr', 'Enkättext', 'Kodlista']]
Variabler = Variabler.rename({'Namn': 'Variabel'}, axis = 'columns')

Värden i **Variabel** ska vara unika.

In [23]:
Variabler.drop_duplicates(['Variabel'], inplace = True)

***  
## <a id = "3.2">3.2 Frekvenstabeller</a>  
Kolumnerna är **V_id**, **Variabel**, **Enkät**, **Kod** och **Frekvens**.

In [24]:
Frekvenstabeller = df[['Namn', 'Enkät', 'Kod', 'Frekvens']]
Frekvenstabeller = Frekvenstabeller.rename({'Namn': 'Variabel'}, axis = 'columns')

**V_id** är huvudnyckel i tabellen, därför räcker det att döpa om index.

In [25]:
Frekvenstabeller.index.name = 'V_id'

***  
## <a id = "3.3">3.3 Enkättexter</a>  
Består endast av **Enkättext** och **Källa**. Eventuellt bör **Enkättext** göras till index eftersom den är huvudnyckel i tabellen.

In [26]:
Enkättexter = df[['Enkättext', 'Källa']]
Enkättexter.set_index('Enkättext', inplace = True)

***  
## <a id = "3.4">3.4 Kodlistor</a>  
Består av **Kodlista** (nyckel) samt **From.1** och **Tom.1**. Notera att de sista klolumnerna blivit omdöpta automatiskt vid inläsning av *metadata*. 

In [27]:
Kodlistor = df[['Kodlista', 'From.1', 'Tom.1']]
Kodlistor.set_index('Kodlista', inplace = True)

***  
## <a id = "3.5">3.5 Svarstexter</a>  
Består av **Svarstext** (nyckel) samt **From** och **Tom**. Den första kolumnen heter **Text** i *metadata* och döps därför om. 

In [28]:
Svarstexter = df[['Text', 'From', 'Tom']]
Svarstexter.rename(columns = {'Text':'Svarstext'}, inplace = True)
Svarstexter.set_index('Svarstext', inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


***  
## <a id = "3.6">3.6 Kodlistor_stor</a>  
Den här tabellen ska innehålla kodlistornas namn i kombination med deras kod och svarstext. Varje sådan kombination ska endast förekomma en gång. Börja med att skära ut kolumnerna **Kodlista**, **Kod** och **Text** ur *metadata*.

In [29]:
d = df[['Kodlista', 'Kod', 'Text']]  
d.head(5)

,Kodlista,Kod,Text
0,janej1,1,Ja
1,janej1,2,Nej
2,janej1,.,Missing
3,omfattning1,1,Mycket högre
4,omfattning1,2,Något högre


I den här mindre tabellen förekommer vissa kodlistor flera gånger. För att identifiera dubletter behövs en slags räknare. Börja med att lägga till en ny kolumn fylld med `0`. Kalla kolumnen för **K_id**.

In [30]:
d.loc[:, 'K_id'] = 0
d.head(5)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,Kodlista,Kod,Text,K_id
0,janej1,1,Ja,0
1,janej1,2,Nej,0
2,janej1,.,Missing,0
3,omfattning1,1,Mycket högre,0
4,omfattning1,2,Något högre,0


**K_id** ska vara unik för varje kombination. Loopen nedan ökar **K_id** med `1` varje gång den passerar en kombination. En kombination identifieras genom att dess sista rad alltid har svarstexten `Missing`.

In [31]:
for id in range(1, len(d['K_id'])):
    if d.iloc[id-1, 2] == 'Missing':
        d.iloc[id: , 3] = d.iloc[id: , 3] + 1

Loopen nedan lägger varje kodlista i listan `behåll` endast en gång. Varje gång en kodlista läggs i `behåll` lägger loopen tillhörande **K_id** i listan `unikid`. Denna andra lista innehåller alltså alla unika förekomster av kombinationer.

In [32]:
unikid = []
behåll = []
for id in range(len(d['K_id'])):
    if d.iloc[id, 0] not in behåll:
        unikid.append(d.iloc[id, 3])
        behåll.append(d.iloc[id, 0])
        

Tills sist skapas tabellen *Kodlistor_stor* genom att endast ha med de rader för vilka **K_id** återfinns i listan `unikid`.

In [33]:
Kodlistor_stor = d.loc[d['K_id'].isin(unikid)]

Gör om **K_id** till ett index så att den kan agera nyckel i tabellen. 

In [34]:
Kodlistor_stor.loc[:, 'K_id'] = Kodlistor_stor.index
Kodlistor_stor.set_index('K_id', inplace = True)

***  
<span style="color:dodgerblue; font-family:freestyle script; font-size:3em">SKISS</span>

Variabler för vilka enkät = `w+n` och dubbelkodning = `Nan` har två frekvenstabeller efter varandra i radledd. I det här avsnittet kommer sådana frekvenstabeller, som alltså ligger till höger om (i kolumnledd) variabeln ifråga, att flyttas ner. Variabeln kommer att kopieras och infogas direkt under sig själv. I det övre läget kommer endast frekventabell för arbetande att förekomma, och i det undre ska frekvenstabellen för icke-arbetande ligga.  

<img src="bild.png" style="width:1000px;height:400px"/>

### Förkortningar
* $ V = $ kombination av rader med samma variabelnamn samt **Enkät** $ = $ `w+n` och **Dubbelkodning** $ = $ `NaN`
* $ v = $ första raden i något $ V $  
* $D = V + D'$  
* $D' = D - V$  
* $d = $ första raden i något $ D $  
* $DV = D + V$

Läs in *metadata* i en ny `DataFrame` eftersom **Enkät** har blivit ändrad. Identifiera radnummer för de variabler som har **Enkät** = `w+n` samt **Dubbelkodning** = `NaN`. Varje kombination av rader som uppfyller detta villkor och som har samma variabelnamn kallas för $V$.

In [35]:
df1 = pd.read_excel('Metadata.xlsx')

df2 = df1[(df1.loc[:, 'Enkät'] == 'w+n') & (df1.loc[:, 'Dubbelkodning'].isna() == True)]

Notera att avståndet mellan index för dessa rader inte längre måste vara 1 (som i ett vanligt index). Skapa därför en ny kolumn och lägg index till de utvalda raderna där, det blir enklare att loopa över ett "vanligt" index (0, 1, 2, 3, 4, ...) senare.

In [36]:
df2['id'] = df2.index

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Välj ut det första radnummret $v$ för varje $V$. I den här loopen refererar kolumn 0 till **Namn** och -1 till **id**.

In [37]:
namn = []
v = []

for i in df2.index:
    if df2.loc[i, 'Namn'] not in namn:
        namn.append(df2.loc[i, 'Namn'])
        v.append(df2.loc[i, 'id'])

Låt $n$ vara antalet $V$. Tabellen *metadata* måste delas upp i $n$ delar ($D$). Varje $D$ består av $V$ samt $D' = D-V$. Varje $D$ har en första rad som kallas $d$. Ifall två $V$ följer efter varandra i *metadata* kommer $D'$ vara tomt. För vissa $D$ kommer alltså $d$ och $v$ sammanfalla.  
Den första $d$ i *metadata* är såklart `0`. Välj ut resterande $d$ genom att hitta den första rad efter $v$ i vilken variabelnamnet ändras.  

In [38]:
d = [0]

for i in v:
    for j in range(i+1, len(df1.loc[:, 'Namn'])):
        if df1.loc[j, 'Namn'] != df1.loc[j-1, 'Namn']:
            d.append(j)
            break

Nu finns en lista med samtliga $d$. Skapa en lista `e` som är precis som `d` fast utan det första elementet och med ett extra element i slutet. varje par $(d, e)$ utgör nu första respektive sista rad i något $D$.

In [39]:
e = d[1:]
e.append(len(df1.iloc[:, 0]))

Skär ut samtliga $D$ och lägg i en lista.

In [40]:
D = [df1.iloc[i:j, :] for i, j in zip(d, e)]

Gå igenom samtliga $D$ och ersätt samtliga värden i frekvenstabellerna i listan `NONWORKFREK` med `NaN`.

In [41]:
for i, j in zip(D, v):
    for k in NONWORKFREK:
        i.loc[j:, k] = np.nan        

På samma sätt utgör $(v, e)$ första respektive sista rad i $V$. Skär ut samtliga $V$ och lägg i en lista.

In [42]:
V = [df1.iloc[i:j, :] for i, j in zip(v, e)]

Gå igenom samtliga $V$ och ersätt samtliga värden i frekvenstabellerna listan `WORKFREK` med `NaN`.

In [43]:
for i in V:
    for j in WORKFREK:
        i.loc[:, j] = np.nan

Nu ska varje $D$ sammanfogas i radledd med sitt tillhörande $V$. Spara resultatet i listan `DV` .

In [44]:
DV = [pd.concat([i, j]) for i, j in zip(D, V)]

Notera att det verkar finnas, längst ner i *metadata*, ett stycke $D$ som inte har något $V$. Detta går att se genom att jämföra längden på listan `D` med listan `V`. Listan `D` har ett element mer än `V`. Eftersom listorna är av olika längd kommer det sista elementet i `D` att undantas från sammanslagning i föregående rad. Lägg därför detta element från `D` längst bak i `DV`.

In [45]:
DV.append(D[-1])

Sammanfonga nu samtliga element i `DV` i radledd och skapa ett nytt index.

In [46]:
df3 = pd.concat(DV, ignore_index = True)